In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-10 13:39:57.031538: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 13:39:57.132265: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 13:39:57.132278: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 13:39:57.152706: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 13:39:57.671843: W tensorflow/stream_executor/platform/de

In [2]:
name_all=[]
auc_roc=[]
name='all_algorithm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100


2022-11-10 13:40:02.249586: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 13:40:02.250292: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:40:02.250359: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-10 13:40:02.250439: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-10 13:40:02.250499: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 393ms/step - loss: 3.9825 - auc: 0.2616
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 2.4301 - auc: 0.8410
Epoch 3/100
1/1 [==============================] - 0s 37ms/step - loss: 2.4056 - auc: 0.9148
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 2.5164 - auc: 0.9217
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 2.5947 - auc: 0.9177
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 2.5182 - auc: 0.9214
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 2.4277 - auc: 0.9229
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 2.2965 - auc: 0.9337
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 2.1287 - auc: 0.9425
Epoch 10/100
1/1 [==============================] - 0s 30ms/step - loss: 1.9408 - auc: 0.9568
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - loss: 1.7684 

1/1 [==============================] - 0s 28ms/step - loss: 0.1547 - auc: 0.9983
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1566 - auc: 0.9985
Epoch 90/100
1/1 [==============================] - 0s 30ms/step - loss: 0.1549 - auc: 0.9980
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1564 - auc: 0.9979
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1485 - auc: 0.9988
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1488 - auc: 0.9986
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1510 - auc: 0.9974
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1465 - auc: 0.9983
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1487 - auc: 0.9978
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1459 - auc: 0.9982
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [3]:
name='remove_svm'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','rf']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 333ms/step - loss: 2.4415 - auc: 0.6702
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 2.4088 - auc: 0.8406
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 2.2410 - auc: 0.8745
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.9947 - auc: 0.8987
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.7552 - auc: 0.9251
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6507 - auc: 0.9321
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5972 - auc: 0.9295
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.4818 - auc: 0.9538
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3569 - auc: 0.9771
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2603 - auc: 0.9876
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.1472 - auc: 0.9978
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1429 - auc: 0.9978
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1490 - auc: 0.9970
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1427 - auc: 0.9980
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1398 - auc: 0.9983
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1430 - auc: 0.9978
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1456 - auc: 0.9976
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1390 - auc: 0.9979
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1376 - auc: 0.9982
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1421 - auc: 0.9978
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [4]:
name='remove_rf'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','dmpnn','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 335ms/step - loss: 3.2618 - auc: 0.2836
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 2.2787 - auc: 0.8001
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 2.3608 - auc: 0.8624
Epoch 4/100
1/1 [==============================] - 0s 36ms/step - loss: 2.4451 - auc: 0.8769
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 2.3875 - auc: 0.8878
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 2.2610 - auc: 0.9010
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 2.0902 - auc: 0.9136
Epoch 8/100
1/1 [==============================] - 0s 38ms/step - loss: 1.8631 - auc: 0.9307
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.6779 - auc: 0.9434
Epoch 10/100
1/1 [==============================] - 0s 26ms/step - loss: 1.5339 - auc: 0.9591
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.1669 - auc: 0.9978
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1679 - auc: 0.9977
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1640 - auc: 0.9977
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1688 - auc: 0.9975
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1733 - auc: 0.9960
Epoch 93/100
1/1 [==============================] - 0s 23ms/step - loss: 0.1649 - auc: 0.9974
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1606 - auc: 0.9979
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1618 - auc: 0.9976
Epoch 96/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1649 - auc: 0.9975
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1599 - auc: 0.9975
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [5]:
name='remove_dmpnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','lr','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 335ms/step - loss: 2.5790 - auc: 0.5681
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 2.3097 - auc: 0.8434
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 2.3367 - auc: 0.8699
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 2.1525 - auc: 0.8966
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.9435 - auc: 0.9169
Epoch 6/100
1/1 [==============================] - 0s 33ms/step - loss: 1.7328 - auc: 0.9353
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5872 - auc: 0.9523
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5222 - auc: 0.9554
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4574 - auc: 0.9602
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3721 - auc: 0.9700
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.1370 - auc: 0.9985
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.1424 - auc: 0.9984
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1400 - auc: 0.9984
Epoch 91/100
1/1 [==============================] - 0s 44ms/step - loss: 0.1468 - auc: 0.9972
Epoch 92/100
1/1 [==============================] - 0s 47ms/step - loss: 0.1445 - auc: 0.9980
Epoch 93/100
1/1 [==============================] - 0s 38ms/step - loss: 0.1412 - auc: 0.9981
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1415 - auc: 0.9977
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1420 - auc: 0.9977
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1403 - auc: 0.9982
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1417 - auc: 0.9980
Epoch 98/100
1/1 [==============================] - 0s 41ms/step - loss: 

In [6]:
name='remove_lr'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','fcnn','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 346ms/step - loss: 2.5584 - auc: 0.5736
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 2.2872 - auc: 0.8557
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 2.3162 - auc: 0.8775
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 2.1640 - auc: 0.8995
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.9431 - auc: 0.9203
Epoch 6/100
1/1 [==============================] - 0s 23ms/step - loss: 1.7403 - auc: 0.9387
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5812 - auc: 0.9595
Epoch 8/100
1/1 [==============================] - 0s 36ms/step - loss: 1.5228 - auc: 0.9610
Epoch 9/100
1/1 [==============================] - 0s 46ms/step - loss: 1.4766 - auc: 0.9594
Epoch 10/100
1/1 [==============================] - 0s 43ms/step - loss: 1.4142 - auc: 0.9641
Epoch 11/100
1/1 [==============================] - 0s 42ms/step - l

1/1 [==============================] - 0s 23ms/step - loss: 0.1445 - auc: 0.9976
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1453 - auc: 0.9974
Epoch 90/100
1/1 [==============================] - 0s 41ms/step - loss: 0.1449 - auc: 0.9980
Epoch 91/100
1/1 [==============================] - 0s 45ms/step - loss: 0.1398 - auc: 0.9981
Epoch 92/100
1/1 [==============================] - 0s 47ms/step - loss: 0.1440 - auc: 0.9976
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1393 - auc: 0.9982
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1417 - auc: 0.9980
Epoch 95/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1411 - auc: 0.9980
Epoch 96/100
1/1 [==============================] - 0s 34ms/step - loss: 0.1476 - auc: 0.9976
Epoch 97/100
1/1 [==============================] - 0s 46ms/step - loss: 0.1389 - auc: 0.9980
Epoch 98/100
1/1 [==============================] - 0s 37ms/step - loss: 

In [7]:
name='remove_fcnn'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['xgb','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 0s 406ms/step - loss: 2.4843 - auc: 0.6186
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 2.2315 - auc: 0.8671
Epoch 3/100
1/1 [==============================] - 0s 43ms/step - loss: 2.2257 - auc: 0.8943
Epoch 4/100
1/1 [==============================] - 0s 45ms/step - loss: 2.0737 - auc: 0.9172
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.8635 - auc: 0.9364
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.6764 - auc: 0.9526
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5186 - auc: 0.9738
Epoch 8/100
1/1 [==============================] - 0s 37ms/step - loss: 1.4539 - auc: 0.9763
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3974 - auc: 0.9776
Epoch 10/100
1/1 [==============================] - 0s 48ms/step - loss: 1.3378 - auc: 0.9780
Epoch 11/100
1/1 [==============================] - 0s 49ms/step - l

1/1 [==============================] - 0s 38ms/step - loss: 0.1256 - auc: 0.9986
Epoch 89/100
1/1 [==============================] - 0s 39ms/step - loss: 0.1348 - auc: 0.9981
Epoch 90/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1280 - auc: 0.9987
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.1306 - auc: 0.9980
Epoch 92/100
1/1 [==============================] - 0s 32ms/step - loss: 0.1332 - auc: 0.9977
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1255 - auc: 0.9983
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.1315 - auc: 0.9983
Epoch 95/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1271 - auc: 0.9983
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.1332 - auc: 0.9978
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1330 - auc: 0.9981
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [8]:
name='remove_xgb'
list_f=['AtomPairFP','Connectivity','Charge',
       'Kappa','MAP4','MHFP6','MOE','MorganFP','Topology','TorsionFP']
list_m=['fcnn','lr','dmpnn','rf','svm']
main_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


external_all_feature=pd.DataFrame()
for feature in list_f:
    for m in list_m:
        read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
        df_main=pd.DataFrame(read_main)
        pred1=[]
        for p in df_main['pred']:
            if p<=0.5:
                p=0
            else:
                p=1
            pred1.append(p)
        external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)


read_m = pd.read_csv("main_set_plus_"+feature+".csv")
df_m = pd.DataFrame(read_m)
a1 = df_m['active']
c1=[]
for b1 in a1:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_m=pd.DataFrame()
y_m.insert(loc=0, column='active', value=a1)
y_m.insert(loc=1, column='inactive', value=c1)
X_m=main_all_feature


read_e = pd.read_csv("external_set_plus_"+feature+".csv")
df_e = pd.DataFrame(read_e)
a2 = df_e['active']
c1=[]
for b1 in a2:
    if b1==1:
        b1=0
    else:
        b1=1
    c1.append(b1)
y_e=pd.DataFrame()
y_e.insert(loc=0, column='active', value=a2)
y_e.insert(loc=1, column='inactive', value=c1)
X_e=external_all_feature
batch_size = 8192
num_classes = 2
model = Sequential()
model.add(Dense(800,
                activation='relu',
                input_dim=len(X_m.columns),
                kernel_initializer = initializers.RandomNormal(stddev=0.02),
                bias_initializer = initializers.RandomNormal(mean=1.),
                kernel_regularizer = regularizers.l2(1e-1)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['AUC'])
model.fit(X_m, y_m,
          batch_size=batch_size,
          epochs=100)
score = model.evaluate(X_e, y_e, verbose=0)
print(score[1])
name_all.append(name)
auc_roc.append(score[1])
model=''

Epoch 1/100
1/1 [==============================] - 1s 535ms/step - loss: 3.4631 - auc: 0.2651
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 2.1585 - auc: 0.8149
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 2.1953 - auc: 0.8839
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 2.3205 - auc: 0.8967
Epoch 5/100
1/1 [==============================] - 0s 26ms/step - loss: 2.3462 - auc: 0.8993
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 2.2245 - auc: 0.9128
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 2.1426 - auc: 0.9183
Epoch 8/100
1/1 [==============================] - 0s 40ms/step - loss: 1.9725 - auc: 0.9291
Epoch 9/100
1/1 [==============================] - 0s 44ms/step - loss: 1.7916 - auc: 0.9413
Epoch 10/100
1/1 [==============================] - 0s 41ms/step - loss: 1.6354 - auc: 0.9512
Epoch 11/100
1/1 [==============================] - 0s 41ms/step - l

1/1 [==============================] - 0s 40ms/step - loss: 0.1635 - auc: 0.9984
Epoch 89/100
1/1 [==============================] - 0s 43ms/step - loss: 0.1655 - auc: 0.9971
Epoch 90/100
1/1 [==============================] - 0s 37ms/step - loss: 0.1645 - auc: 0.9978
Epoch 91/100
1/1 [==============================] - 0s 22ms/step - loss: 0.1677 - auc: 0.9975
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.1667 - auc: 0.9967
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.1594 - auc: 0.9978
Epoch 94/100
1/1 [==============================] - 0s 31ms/step - loss: 0.1627 - auc: 0.9977
Epoch 95/100
1/1 [==============================] - 0s 39ms/step - loss: 0.1609 - auc: 0.9972
Epoch 96/100
1/1 [==============================] - 0s 41ms/step - loss: 0.1525 - auc: 0.9984
Epoch 97/100
1/1 [==============================] - 0s 36ms/step - loss: 0.1552 - auc: 0.9981
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [9]:
de=pd.DataFrame()
de['name']=name_all
de['auc_roc']=auc_roc
de.to_csv('6model_remove_one_model.csv',index=False)
print(de)

            name   auc_roc
0  all_algorithm  0.934633
1     remove_svm  0.940661
2      remove_rf  0.938394
3   remove_dmpnn  0.932317
4      remove_lr  0.932200
5    remove_fcnn  0.935594
6     remove_xgb  0.932345
